## Cleaning and merging dataframe of different countries

This notebook concat information of each user data to one single dataframe

In [1]:
import pandas as pd
from glob import glob
import os


In [2]:
countries = [
    'belgium',
    'malaysia',
    'indonesia',
    'australia',
    'new-zealand',
    'singapore',
    'united-states',
    'canada',
    'united-kingdom'
]

In [3]:
country_region = {
    'belgium' : 'europe',
    'malaysia': 'asia',
    'indonesia': 'asia',
    'australia': 'oceania',
    'new-zealand': 'oceania',
    'singapore': 'asia',
    'united-states': 'america',
    'canada': 'america',
    'united-kingdom': 'europe'
}

In [4]:
def clean_pd(df, country):
    df['hour'] = pd.to_datetime(df['time']).dt.hour
    df['week'] = pd.to_datetime(df['date']).dt.day_name()
    weekdays = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    hours = pd.DataFrame(pd.DataFrame(pd.get_dummies(df['hour']), columns=range(24)).fillna(0).sum()).T.astype(int)
    days = pd.DataFrame(pd.DataFrame(pd.get_dummies(df['week']), columns=weekdays).fillna(0).sum()).T.astype(int)
    final = pd.concat([hours,days], axis=1)
    final['username'] = df['username'][0]
    final['country'] = country
    final['region'] = country_region[country]
    final['tweets'] = ''.join([t for t in df['tweet']])
    final = final[list(final.columns[31:]) + list(final.columns[:31])]
    return final

In [5]:
combined = pd.concat([pd.concat([clean_pd(pd.read_csv(f, encoding='utf-8'), country) 
                          for f in glob(f'data/{country}/*.csv')]) for country in countries])

In [6]:
combined.shape

(8789, 35)

In [7]:
combined.to_csv('data/data.csv', index=False, encoding='utf-8')